 # FT-ICR analysis - PART 4

Created on 18 April 2020 for the Pentatrap experiment
 
@author: Menno Door<br>
@contact: door+fticr@mpi-k.de<br>
@license: MIT license
 
### Introduction
 
This part allows you to evaluate notebooks with multiple settings/parameters in a batch processing kind of way.
 
### Requirements:
 
The following code was written in Python 3.7/3.8. The required libraries are listed below with a rough description for their task in the code.
 
     papermill (batch processing, https://github.com/nteract/papermill)


-----------------------------------------------------------------------------------------------------------------------
### N O T E: I M P O R T A N T :

This is not a very user friendly thing here... sorry for that, but its really not that often needed. 
Also for merging and plotting in the end, that is all very specific to the parameters looped over in the batch analysis,
so dont expect it to work if you changed to other parameters...

papermill needs the most recent version of jupyter notebook

-----------------------------------------------------------------------------------------------------------------------

In [1]:
### --------------------------------------------------------------------------------------------------------------------------------------
### Imports
from pathlib import Path
from pprint import pprint
import numpy as np
import pandas as pd
import papermill as pm
import os, time
from datetime import datetime
import plotly.express as px
from fticr_toolkit import *

from IPython.core.display import display, HTML


### ------------------------------------------------------------------------------------------------------------------------------------


In [2]:
main_folder_path = "G:\\Yb\\17241+42+_binding_var\\"
main_folder_path = "G:\\Ca\\48Ca_1st_run\\"
#main_folder_path = "G:\\Be\\"
main_folder_path = "G:\\Yb\\172174_var3_opti\\"
#main_folder_path = "G:\\Yb\\172174_var1\\"
#main_folder_path = "F:\\Be_C_sys_PnAPnP\\"
main_folder_path = "F:\\Ne\\"
main_folder_path = "F:\\Be\\"

split_char = "_"
number_idx = -1

do_part1 = False
do_part2 = True

#skip_numbers = [10,11,12,13,14,14,15,16]
#skip_numbers = []
#skip_numbers = [1,2,4,7,10,11,12,14,15,16,18,19]
min_number = 0

In [3]:

### --------------------------------------------------------------------------------------------------------------------------------------
### Example parameter cell from part 2 notebook, the tag is needed for papermill!, this is cell 3

###   P A R A M E T E R   L I S T   ( this is especially used for batch processing using the papermill package but its also nice to have thesse parameters all at one place )
measurement_folder = "G:\\Yb\\\172174_var1\\174Yb42+_172Yb42+_174Yb42+_var3_33_opti"

input_folder = "./part1_data/" # the measurements subfolder where the pre-analysed data (axial frequencies and nu_p phases) is
output_folder = "./results/" # subfolder where the results of this analysis should go
use_settings_file = True # uses the settings file to overwrite this stuff here:

settings = {
    #"grouping": [0,-1,0,-1,0,-1,0,-1,0,-1], # use -1 to not use the axial
    #"grouping": [0,0,0,1,1,1,1,2,2,2], # use -1 to not use the axial
    #"grouping": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], # use -1 to not use the axial
    "grouping": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], # use -1 to not use the axial
    "reuse_averages": True,
    "reuse_fitdata": False,
    "single_axial_fits": False,
    #"make_filter_settings_file": 'filter_settings.csv',
    "make_filter_settings_file": 'filter_settings_auto.csv',
    "fixed_phase_readout": True,
    "filter_axial_by_phase": False,
    "input_file": False,
    "downsample": 5,# axial resonator
    "fit_func": 'pavel', # or daniel
    "plot": True,
    "filter_settings": None, # pandas dset with mc, trap, position, min_cycle, max_cycle; None: try to get filter settings from loaded data; False: no filter applied
    #"filter_settings": pd.DataFrame(
    #    columns=["mcycle", "trap", "position", "min_cycle", "max_cycle"],
    #    data=[[1,2,"position_1", 0, 26],[1,2,"position_2", 0, 26],[1,3,"position_1", 0, 26],[1,3,"position_2", 0, 26]]
    #          ),
    "post_unwrap": False, # if this is True, the analysis will automatically choose the post-unwrap (the pre-unwrap of the next main cycle) for nu_p frequency determination. Maybe the whole post-unwrap part is commented out in step 5, please check.
    "unwrap_range": 10,
    "phase_error_undrift": True, # makes everything worse supprisingly
    "phase_Nmatch_phases": 50, # how many phases of the main cycle to fit and extrapolate to N determination
    "mean_ref_phase": False, # BUGGED!!!! DONT USE! if True, the reference phase will be averaged and the same value substracted for all long phases (not in Ndet), otherwise every single measured reference phase is substraced from the respective long phase
    "phase_filter": True, # filter measured phases by 3 sigma filter inside subcycle
    "single_axial": False, # use single spec no average axial data
    "average": True, # averaging subcycle data to match average_idx from part 1
    "nu_z_from_config": False, # default: False, if float, the config nu_z will be taken and the float value is used as the error on the config nu_z
    "fill_nu_z_from_config": False, #0.08, # default: False, if float, the config nu_z will be taken if the measured nu_z is off by hardcoded 100 Hz (which also includes no value at all) and the float value is used as the error on the config nu_z
    "sideband": False, # use sideband relation to calculate nu_c
    "nu_m_from_unwrap": False, # get the magnetron freq from the unwrap measurements
    "nu_m2_from_theory": True, # use the magnetron freq of one position to calculate the magnetron of the other position, keeping the difference right, common offset doesn't matter
    "nu_z2_from_theory": False, # use the magnetron freq of one position to calculate the magnetron of the other position, keeping the difference right, common offset doesn't matter
    "polydegrees": 'auto', # number of degree of the polynom fit
    "polygrouping": 0, #'auto', # group sizes for the polynom fit
    "poly_mode": "polyfit_fast", # routine for polynom fitting
    "poly_criterion": "AICc", # criterion for best model/poly-degree
    "invert": False,
    "fit_settings": {
        "trap2" : {
            "res_span" : 3000,
            "dip_span": 80,
        },
        
        "trap3" : {
            "res_span" : 3000,
            "dip_span": 80,
        }
    }
    
}

### Main looping over parameters and executing notebooks

In [4]:

### --------------------------------------------------------------------------------------------------------------------------------------
### Main folders with all the measurements

masterfolder = Path(main_folder_path)
batch_results = Path.joinpath( masterfolder, "batch_results_"+datetime.now().strftime("%Y%m%d_%H%M%S")+"/" )
#batch_results = Path.joinpath( masterfolder, "batch_results_20210426_003859/" )
os.mkdir(batch_results)

meas_folders = [x for x in masterfolder.iterdir() if (x.is_dir() and 
                                                      not str(x.stem).startswith("_") and 
                                                      not str(x.stem).startswith("batch") and
                                                      not str(x.stem).startswith("crap") and
                                                      not str(x.stem).endswith("crap")
                                                     )]
pprint(meas_folders)


[WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_10'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_11'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_13'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_14'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_15'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_17'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_18'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_19'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_23'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_6'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_6B'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_7'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_7B'),
 WindowsPath('F:/Ne/12C6+_20Ne10+_12C6+_8')]


In [5]:
# check freedom before...

#table = pd.DataFrame()
#for n_min in [4,5,6,7,0]:                # calculating total time: 5
#    paras["grouping_n_min"] = n_min
#
#    for poly_deg in [3,4,5,6,7,8,9]:   # 5*7 = 35
#        freedom = 2*n_min - poly_deg
#        if freedom < 4 and n_min != 0:   # a little less... ~ 30 * (#meas = 6) = 180 * (mean(t_eval) = 90s) = ~180min = 3h
#            continue
#            
#        this = pd.DataFrame(data=[[n_min, poly_deg, freedom]], columns=["n_min", "polyd", "f"])
#        table = table.append(this)
#print(table)
#print(len(table))

In [6]:
paras = {
    "use_settings_file": True,
    "settings": settings
}

In [7]:


for meas_dir in meas_folders:
    # the measurement folder is needed for the step results and saving the final notebook
    paras["measurement_folder"] = str(meas_dir)
    print(meas_dir)

    # only the good measurements
    '''
    try:
        #if int(str(meas_dir).rsplit("_")[-1]) > 22: # only specific measurements
        if int(str(meas_dir).rsplit(split_char)[number_idx]) < min_number: # only specific measurements
            print("skip...")
            continue
        if int(str(meas_dir).rsplit(split_char)[number_idx]) in skip_numbers: # only specific measurements
            print("skip...")
            continue
    except:
        #raise
        continue
    '''

    ## P A R T  1
    if do_part1:
        
        print("running PART 1", meas_dir)

        paras["input_file"] = 'pnp_dip_unwrap.fhds'
        paras["output_folder"] = "./part1_data/"

        #pprint(paras)

        try:
            os.mkdir(str(meas_dir)+"/part1_data/")
        except:
            pass

        ## Execute notebook
        output_notebook = str(meas_dir)+"/part1_data/part1.ipynb"

        try:
            pm.execute_notebook(
                'Analysis_PART1.ipynb',
                output_notebook,
                parameters = paras,
                nest_asyncio=True
            )

        except Exception as e:
            print('part 1 failed, still try part 2 though...')
            print(e)
        finally:
            try:
                print("convert to HTML ... ")
                os.system("jupyter nbconvert --to HTML "+ output_notebook)
                time.sleep(2)
            except:
                print('oh man... no HTML copy :/')            
        #os.remove(output_notebook)
        #print("delete temporaly notebook... done!")

    # P A R T  2
    if do_part2:
        
        print("running PART 2", meas_dir)
        
        paras["input_folder"] = "./part1_data/"
        paras["output_folder"] = "./results/"
        #paras['settings']["grouping"] = [5] # use -1 to not use the axial

        try:
            os.mkdir(str(meas_dir)+"/results/")
        except:
            pass

        try:
            ## Execute notebook
            output_notebook = str(meas_dir)+"/results/part2.ipynb"
            #print(output_notebook)
            pm.execute_notebook(
                'Analysis_PART2.ipynb',
                output_notebook,
                parameters = paras,
                nest_asyncio=True
            )

            print("convert to HTML ... ")
            os.system("jupyter nbconvert --to HTML "+ output_notebook)
            time.sleep(2)
        except Exception as e:
            print('oh man... part 2 failed! :/ ')
            print(e)
        finally:
            try:
                print("convert to HTML ... ")
                os.system("jupyter nbconvert --to HTML "+ output_notebook)
                time.sleep(2)
            except:
                print('oh man... no HTML copy :/')
        #os.remove(output_notebook)
        #print("delete temporaly notebook... done!")


'''
### Summary:
print("running PART 3")
paras["output_folder"] = str( batch_results )
paras["input_folder"] = str( masterfolder )
output_notebook = str( batch_results )+"/batch.ipynb"
pm.execute_notebook(
    'Analysis_PART3.ipynb',
    output_notebook,
    parameters = paras,
    nest_asyncio=True
)

print("convert to HTML ... ")
os.system("jupyter nbconvert --to HTML "+ output_notebook)
time.sleep(5)
#os.remove(output_notebook)
#print("delete temporaly notebook... done!")
'''

F:\Ne\12C6+_20Ne10+_12C6+_10
running PART 2 F:\Ne\12C6+_20Ne10+_12C6+_10


Executing:   0%|          | 0/88 [00:00<?, ?cell/s]

convert to HTML ... 
convert to HTML ... 
F:\Ne\12C6+_20Ne10+_12C6+_11
running PART 2 F:\Ne\12C6+_20Ne10+_12C6+_11


Executing:   0%|          | 0/88 [00:00<?, ?cell/s]

convert to HTML ... 
convert to HTML ... 
F:\Ne\12C6+_20Ne10+_12C6+_13
running PART 2 F:\Ne\12C6+_20Ne10+_12C6+_13


...

Executing:   0%|          | 0/88 [00:00<?, ?cell/s]

oh man... part 2 failed! :/ 

---------------------------------------------------------------------------
Exception encountered at "In [52]":
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-52-754a3c036010> in <module>
      6 unique_traps = step7_results.trap.unique()
      7 if len(unique_traps) < 2:
----> 8     raise KeyError('not enough trap data for cancellation, only traps: '+str(unique_traps))
      9 
     10 # ONLY WITH 2 TRAP DATA!!!

KeyError: 'not enough trap data for cancellation, only traps: [2]'

convert to HTML ... 
F:\Ne\12C6+_20Ne10+_12C6+_8
running PART 2 F:\Ne\12C6+_20Ne10+_12C6+_8


Executing:   0%|          | 0/88 [00:00<?, ?cell/s]

convert to HTML ... 
convert to HTML ... 


'\n### Summary:\nprint("running PART 3")\nparas["output_folder"] = str( batch_results )\nparas["input_folder"] = str( masterfolder )\noutput_notebook = str( batch_results )+"/batch.ipynb"\npm.execute_notebook(\n    \'Analysis_PART3.ipynb\',\n    output_notebook,\n    parameters = paras,\n    nest_asyncio=True\n)\n\nprint("convert to HTML ... ")\nos.system("jupyter nbconvert --to HTML "+ output_notebook)\ntime.sleep(5)\n#os.remove(output_notebook)\n#print("delete temporaly notebook... done!")\n'